# Лабораторна №2. Потьомкін Лев, К-27

## 1. Метод Гауса з вибором головного елемента

### Теорія

СЛАР - система лінійних алгебраїчних рівнянь, вигляду

\begin{cases} a_{11}x_1 + a_{12}x_2 + ... + a_{1n}x_n = b_1 \\ a_{21}x_1 + a_{22}x_2 + ... + a_{2n}x_n = b_2 \\ ... \\ a_{n1}x_1 + a_{n2}x_2 + ... + a_{nn}x_n = b_n \end{cases} 

Її можна представити у вигляді добутку матриці коефіцієнтів на вектор змінних: $Ax = b$

Для розв'язку СЛАР можна застосувати як прямі, так і ітераційні методи. Поширеним прямим способом розв’язання систем лінійних рівнянь є алгоритм послідовного виключення невідомих, що має назву метод Гауса.
   
Існують різні алгоритми його реалізації. Один із них - метод Гауса із вибором головного елемента - розглядається у цій роботі.
   
Метод Гауса із вибором головного елемента передбачає послідовність однотипних кроків виключення невідомих із системи рівнянь. На кожному кроці виключається чергове невідоме за допомогою рівняння з найбільшим за модулем (або хоча б ненульовим) коефіцієнтом при відподвідному невідомому.

In [76]:
def gauss(A, eq=0, var=0):
    n, m = len(A), len(A[0])-1
    
    # дійшли до кінця
    if eq == n or var == m:
        if any(A[k][m] != 0 for k in range(eq, n)):
            return None # немає розв'язку
        
        if n >= m and A[m-1][m-1] != 0:
            return [A[k][m] for k in range(m)]
        
        return None # нескінченна к-ть розв'язків
    
    # обираємо головний елемент
    for k in range(eq, n):
        if A[k][var] != 0:
            A[eq], A[k] = A[k], A[eq]
            break
    
    # якщо ненульових немає, переходимо до наступного р-ння
    if A[eq][var] == 0:
        return gauss(A, eq, var+1)
    
    # ділимо на головний елемент
    for i in range(var+1, m+1):
        A[eq][i] /= A[eq][var]
    A[eq][var] = 1
    
    # виключаємо невідомі
    for k in range(n):
        if k == eq or A[k][var] == 0: continue
        coef = A[k][var]
        for i in range(var, m+1):
            A[k][i] -= coef*A[eq][i]
      
    # переходимо до наступного р-ння
    return gauss(A, eq+1, var+1)


In [77]:
A_b = [
    [4, 3, 1, 0, 14],
    [-2, 2, 6, 1, 31],
    [0, 5, 2, 3, 33],
    [0, 1, 2, 7, 45]
]

x = gauss(A_b)
Matrix(x)

Matrix([
[1.0],
[2.0],
[4.0],
[5.0]])

## 2. Метод квадратного кореня

### Теорія

Нехай є СЛАР $Ax = b$

Та нехай $A$ - симетрична матриця. Тоді існують такi $S$, $D$ що $ A = S^TDS $, де $S$ - верхньотрикутна матриця, $D$ - діагональна.

Тоді розв'язавши систему

- $S^TDy = b$
- $Sx = y$

розв'яжемо й вихідну СЛАР.

Так як обидві матриці $S^TD$ та $S$ трикутні, треба виконати лише зворотній хід методу Гауса.


In [78]:
from sympy import sign, Matrix

def sqrt_method(A, b):
    # перевірка на симетричність
    assert(A.is_symmetric())
    
    n = len(b)
    S = Matrix.zeros(n)
    D = Matrix.zeros(n)
    
    # обчислюємо матриці S, D
    for i in range(n):
        temp = A[i, i] - sum(S[p, i]**2 * D[p, p] for p in range(i))
        D[i, i] = sign(temp)
        S[i, i] = abs(temp) ** 0.5
        for j in range(i+1, n):
            temp  = A[i, j] - sum(S[p, i] * D[p, p] * S[p, j] for p in range(i))
            S[i, j] = temp / (D[i, i] * S[i, i])
            
    # розв'язуємо SᵀDy = b
    y = solve_triangular(S.T * D, b)
    # розв'язуємо Sx = y
    x = solve_triangular(S[::-1, ::-1], y[::-1])[::-1]
    return x

def solve_triangular(A, b):
    n = len(b)
    x = b.copy()
    
    for i in range(n):
        x[i] /= A[i, i]
        for j in range(i+1, n):
            x[j] -= x[i] * A[j, i]
        
    return x

In [79]:
A = Matrix([
    [1, 2, 0],
    [2, 2, 4],
    [0, 4, 3]
])

b = Matrix([11, 34, 31])
x = sqrt_method(A, b)
Matrix(x)

Matrix([
[3.0],
[4.0],
[5.0]])